In [22]:
from dotenv import dotenv_values
import praw
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [23]:
# Download the VADER lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\seb\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [24]:
# Load environment variables
env = dotenv_values('./.env')

In [25]:
# Set up Reddit API credentials
reddit = praw.Reddit(
    client_id=env['R_CLIENT_ID'],
    client_secret=env['R_CLIENT_SECRET'],
    user_agent=env['R_USER_AGENT']
)

In [26]:
# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

praw.models.Submission: 
https://praw.readthedocs.io/en/v7.7.1/code_overview/models/submission.html

In [27]:
from time import sleep

In [28]:
# Stream post submissions in selected subreddit and analyze sentiment
subreddit_name = 'WallStreetBets'

subreddit = reddit.subreddit(subreddit_name)
submission_data = {}  # submission data storage

for submission in subreddit.stream.submissions():
    if submission.stickied:  # if post is pinned, skip it
        continue
    if not submission.selftext:  # if post doesn't contain text, skip it
        continue
    
    sentiment = sia.polarity_scores(submission.selftext)

    submission_data[f"{submission.id}"] = {
        'num_comments': submission.num_comments,
        'score': submission.score,
        'upvote_ratio': submission.upvote_ratio,
        'sentiment': sentiment['compound']
    }

    print(list(submission_data.items())[-1])  ## DEBUG: print last submission collected

('1esnkj2', {'num_comments': 329, 'score': 1210, 'upvote_ratio': 0.9, 'sentiment': 0.0631})
('1esoav3', {'num_comments': 121, 'score': 242, 'upvote_ratio': 0.89, 'sentiment': 0.6166})
('1espi86', {'num_comments': 11, 'score': 15, 'upvote_ratio': 0.8, 'sentiment': -0.3818})
('1esqryb', {'num_comments': 15, 'score': 0, 'upvote_ratio': 0.31, 'sentiment': 0.6536})
('1esr9gi', {'num_comments': 13510, 'score': 317, 'upvote_ratio': 0.92, 'sentiment': 0.0})
('1ess1mb', {'num_comments': 87, 'score': 0, 'upvote_ratio': 0.47, 'sentiment': 0.9331})
('1esu8wh', {'num_comments': 111, 'score': 407, 'upvote_ratio': 0.9, 'sentiment': 0.3244})
('1esujkz', {'num_comments': 16, 'score': 60, 'upvote_ratio': 0.9, 'sentiment': 0.0})
('1esvd12', {'num_comments': 10, 'score': 45, 'upvote_ratio': 0.92, 'sentiment': -0.5574})
('1esvk0o', {'num_comments': 10, 'score': 3, 'upvote_ratio': 0.62, 'sentiment': 0.5719})
('1eswty1', {'num_comments': 1, 'score': 7, 'upvote_ratio': 0.89, 'sentiment': 0.6908})
('1eswveu', 

KeyboardInterrupt: 